# Model Endpoint Management in AlloyDB for PostgreSQL

> [AlloyDB](https://cloud.google.com/alloydb) is a fully managed PostgreSQL compatible database service for your most demanding enterprise workloads.
AlloyDB combines the best of Google with PostgreSQL, for superior performance, scale, and availability. Extend your database application to build AI-powered
experiences leveraging AlloyDB Langchain integrations.

This notebook goes over how to use Model endpoint management in AlloyDB using the `AlloyDBModelManager` and `AlloyDBEmbeddings` classes.

Model Endpoint Management allows Google Cloud Databases, such as AlloyDB, Cloud SQL, Spanner, etc. to directly invoke Large Language Models (LLMs) within SQL queries, facilitating seamless integration of AI capabilities into data workflows. This feature enables developers to leverage LLM-powered insights in real time, improving the efficiency of data processing tasks.

Learn more about the package on [GitHub](https://github.com/googleapis/langchain-google-alloydb-pg-python/).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-alloydb-pg-python/blob/main/docs/model_endpoint_management.ipynb)

## Before You Begin

To run this notebook, you will need to do the following:

 * [Create a Google Cloud Project](https://developers.google.com/workspace/guides/create-project)
 * [Enable the AlloyDB API](https://console.cloud.google.com/flows/enableapi?apiid=alloydb.googleapis.com)
 * [Create a AlloyDB instance](https://cloud.google.com/alloydb/docs/instance-primary-create)
 * [Create a AlloyDB database](https://cloud.google.com/alloydb/docs/database-create)
 * [Set the google_ml_integration.enable_model_support database flag to on for an instance](https://cloud.google.com/alloydb/docs/instance-configure-database-flags)


### (Optional) Set up non-default database users

First, [Add an IAM database user to the database](https://cloud.google.com/alloydb/docs/manage-iam-authn) or a [custom database user](https://cloud.google.com/alloydb/docs/database-users/about#create).

Second, set up the required user permissions by running the following commands on [AlloyDBStudio](https://cloud.google.com/alloydb/docs/manage-data-using-studio) or any `psql` terminal.


The `google_ml_integration` extension must first be installed.

In [ ]:
CREATE EXTENSION google_ml_integration VERSION '1.3';

Grant permissions for the user to access the tables in the `google_ml_extension`. Replace the `<NON_SUPER_USER>` with your user.

For more information, see [Enabling extension](https://cloud.google.com/alloydb/docs/ai/model-endpoint-register-model#enable-extension).


In [ ]:
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA google_ml TO <NON_SUPER_USER>;

Grant permission for the user to access the `embedding` function of `google_ml_extension`. Replace the `<NON_SUPER_USER>` with your user.

For more information about the above permission, see [Generate embeddings](https://cloud.google.com/alloydb/docs/ai/work-with-embeddings).

In [ ]:
GRANT EXECUTE ON FUNCTION embedding TO <NON_SUPER_USER>;

### 🦜🔗 Library Installation
Install the integration library, `langchain-google-alloydb-pg`. The library must be version v0.8.0 or higher.

In [ ]:
%pip install --upgrade --quiet langchain-google-alloydb-pg langchain-core google-cloud-alloydb-connector[pg8000]

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel.
For Vertex AI Workbench you can restart the terminal using the button on top.

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 Authentication
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

* If you are using Colab to run this notebook, use the cell below and continue.
* If you are using Vertex AI Workbench, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
from google.colab import auth

auth.authenticate_user()

### ☁ Set Your Google Cloud Project
Set your Google Cloud project so that you can leverage Google Cloud resources within this notebook.

If you don't know your project ID, try the following:

* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.
PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### Enable database integration with Vertex AI

To enable database integration with Vertex AI, the AlloyDB service agent (`service-<PROJECT_NUMBER>@gcp-sa-alloydb.iam.gserviceaccount.com`) must be granted the Vertex AI User role. For more information on authentication for Vertex AI, see [this](https://cloud.google.com/alloydb/docs/ai/model-endpoint-register-model#vertex-provider).

In [ ]:
PROJECT_NUMBER=!gcloud projects describe {PROJECT_ID} --format="value(projectNumber)"

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
--member="serviceAccount:service-{PROJECT_NUMBER[0]}@gcp-sa-alloydb.iam.gserviceaccount.com" \
--role="roles/aiplatform.user"

## Set up connection pool

### Set AlloyDB database values
Find your database values, in the [AlloyDB cluster page](https://console.cloud.google.com/alloydb?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687).

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
CLUSTER = "my-alloydb-cluster"  # @param {type: "string"}
INSTANCE = "my-alloydb-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
TABLE_NAME = "vector_store"  # @param {type: "string"}

### AlloyDBEngine Connection Pool

To connect to AlloyDB and use Model endpoint management is an `AlloyDBEngine` object is required. The `AlloyDBEngine` configures a connection pool to your AlloyDB database, enabling successful connections from your application and following industry best practices.

To create a `AlloyDBEngine` using `AlloyDBEngine.from_instance()` you need to provide only 5 things:

1. `project_id` : Project ID of the Google Cloud Project where the AlloyDB instance is located.
1. `region` : Region where the AlloyDB instance is located.
1. `cluster`: The name of the AlloyDB cluster.
1. `instance` : The name of the AlloyDB instance.
1. `database` : The name of the database to connect to on the AlloyDB instance.

By default, [IAM database authentication](https://cloud.google.com/alloydb/docs/connect-iam) will be used as the method of database authentication. This library uses the IAM principal belonging to the [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) sourced from the environment.

Optionally, [built-in database authentication](https://cloud.google.com/alloydb/docs/database-users/about) using a username and password to access the AlloyDB database can also be used. Just provide the optional `user` and `password` arguments to `AlloyDBEngine.from_instance()`:

* `user` : Database user to use for built-in database authentication and login.
* `password` : Database password to use for built-in database authentication and login.

To connect to your AlloyDB instance from this notebook, you will need to enable public IP on your instance. Alternatively, you can follow [these instructions](https://cloud.google.com/alloydb/docs/connect-external) to connect to an AlloyDB for PostgreSQL instance with Private IP from outside your VPC.
Learn more about [specifying IP types](https://github.com/GoogleCloudPlatform/alloydb-python-connector?tab=readme-ov-file#specifying-ip-address-type).


In [ ]:
from langchain_google_alloydb_pg import AlloyDBEngine
from google.cloud.alloydb.connector import IPTypes

engine = await AlloyDBEngine.afrom_instance(
    project_id=PROJECT_ID,
    region=REGION,
    cluster=CLUSTER,
    instance=INSTANCE,
    database=DATABASE,
    ip_type=IPTypes.PUBLIC,
)

## Register a model with `AlloyDBModelManager`
The `AlloyDBModelManager` class allows the user to create, get, list, and drop models. A model is required by the `AlloyDBEmbeddings` class to be used to embed documents on insertion into the vector store and during similarity searches.

Initialize an instance of `AlloyDBModelManager` with the connection pool through the `AlloyDBEngine` object.

In [ ]:
from langchain_google_alloydb_pg import AlloyDBModelManager, AlloyDBModel

model_manager = await AlloyDBModelManager.create(engine)

On creating the `AlloyDBModelManager` object, it will run a prerequisite check to ensure:
* The extension is up to date: `google_ml_integration` extension is installed and the version is greater than 1.3
* The database flag is set: `google_ml_integration.enable_model_support` is set to on.

#### List all models available
This list includes the [pre built models](https://cloud.google.com/alloydb/docs/ai/model-endpoint-register-model#add-vertex) and any other model you may have created.


In [ ]:
results = await model_manager.alist_models()
print(results)

#### Get a specific model
To retrieve a specific model you will have to provide the `model_id` to the `aget_model()` function.

If the model with the specified model_id exists, then the AlloyDBModel dataclass of it is returned.
Otherwise None is returned.

In [ ]:
result = await model_manager.aget_model(model_id="textembedding-gecko")
print(result)

#### Create a custom text embedding model

To create a custom textembedding model you need to pass these parameters to the `acreate_model()` function :

* model_id: A unique ID for the model endpoint that you define.
* model_provider: The provider of the model endpoint (`google` for vertexAI and `custom` for custom  hosted models).
* model_type: The model type (set this value to `text_embedding` for text embedding model endpoints or `generic` for all other model endpoints).
* model_qualified_name: The fully qualified name in case the model endpoint has multiple versions or if the model endpoint defines it.

You can customize your model further with some optional parameters. For all the details and possibilities, check out the [reference doc](https://cloud.google.com/alloydb/docs/reference/model-endpoint-reference#google_mlcreate_model).


**Note**: The `acreate_model()` function doesn't return any value directly.
You'll need to use `alist_models()` or `aget_model()` to verify if your model was created successfully.

In [ ]:
await model_manager.acreate_model(
    model_id="textembedding-gecko@003",
    model_provider="google",
    model_qualified_name="textembedding-gecko@003",
    model_type="text_embedding",
)

**Note**: A model once created can also be dropped.

#### Create custom third-party models
You can also create a third-party custom text embedding model using these steps. You can also create a third-party custom text embedding model, such as Hugging Face models.

For all models except Vertex AI model endpoints, you can store your API keys or bearer tokens in Secret Manager. This step is optional if your model endpoint doesn't handle authentication through Secret Manager.

For information, see [Authentication for custom hosted models](https://cloud.google.com/alloydb/docs/ai/model-endpoint-register-model#set_up_authentication_for_custom-hosted_models).


### Removing a Model
If you no longer need a specific model, you can easily remove it using the adrop_model function by providing the model_id.

To make sure the model has been deleted, you can use the alist_models function to list all your remaining models. The deleted model should no longer appear in the list.

In [ ]:
await model_manager.adrop_model(model_id="textembedding-gecko@003")

## Understanding the AlloyDBModel Dataclass

When you retrieve a model using the function `aget_model()`, you'll receive an `AlloyDBModel` object.

Here's a breakdown of what's inside:
* model_id (str) : A unique ID for the model endpoint that you define.
* model_request_url (Optional[str]) : The model-specific endpoint when adding other text embedding and generic model endpoints.
* model_provider (str) : The provider of the model endpoint. Set to google for Vertex AI model endpoints and custom for custom-hosted model endpoints.
* model_type (str) : The model type. You can set this value to text_embedding for text embedding model endpoints or generic for all other model endpoints.
* model_qualified_name (str) : The fully qualified name in case the model endpoint has multiple versions or if the model endpoint defines it.
* model_auth_type (Optional[str]) : The authentication type used by the model endpoint. You can set it to either alloydb_service_agent_iam for Vertex AI models or secret_manager for other providers.
* model_auth_id (Optional[str]) : The secret ID that you set and is subsequently used when registering a model endpoint.
* input_transform_fn (Optional[str]) : The SQL function name to transform input of the corresponding prediction function to the model-specific input.
* output_transform_fn (Optional[str]) : The SQL function name to transform model specific output to the prediction function output.

See below for an example of AlloyDBModel instance on using `aget_model(model_id="textembedding-gecko@001")`.

In [ ]:
AlloyDBModel(
    model_id="textembedding-gecko@001",
    model_request_url="publishers/google/models/textembedding-gecko@001",
    model_provider="google",
    model_type="text_embedding",
    model_qualified_name="textembedding-gecko@001",
    model_auth_type="alloydb_service_agent_iam",
    model_auth_id=None,
    input_transform_fn="google_ml.vertexai_text_embedding_input_transform",
    output_transform_fn="google_ml.vertexai_text_embedding_output_transform",
)

## Generate Vector Embeddings with `AlloyDBEmbeddings`
The `AlloyDBEmbeddings` class allows users to utilize the in database embedding generation functions available via Model Endpoint Management.

In the below example, we are using the `textembedding-gecko@003` model that we created using the Model Manager.

**Note**: If you have dropped the above model, you can recreate it or use `textembedding-gecko@001`.

In [ ]:
from langchain_google_alloydb_pg import AlloyDBEmbeddings

model_id = "textembedding-gecko@003"
embedding_service = await AlloyDBEmbeddings.create(engine=engine, model_id=model_id)

**Note**: This tutorial demonstrates the async interface. All async methods have corresponding sync methods.

On creating an instance of the `AlloyDBEmbeddings` class, it checks if the model exists.
If the model does not exist with that model_id, the class throws a `ValueError`.

#### Using `AlloyDBEmbeddings` as an embedding service

The `AlloyDBEmbeddings` class can be used as the embedding service with an `AlloyDBVectorStore` to generate embeddings on document insertion and for similarity searches.

Learn more about getting started with the [`AlloyDBVectorStore`](https://github.com/googleapis/langchain-google-alloydb-pg-python/blob/main/docs/vector_store.ipynb).

In [ ]:
import uuid
from langchain_core.documents import Document
from langchain_google_alloydb_pg import AlloyDBVectorStore

# (Optional) Create a new vector store table
VECTOR_SIZE = 768  # For textembeddding-gecko@003 model
await engine.ainit_vectorstore_table(
    table_name="vector_store_table",
    vector_size=VECTOR_SIZE,
    overwrite_existing=True,
)

# Initialize the vector store instance with AlloyDBEmbeddings
vs = await AlloyDBVectorStore.create(
    engine,
    embedding_service=embedding_service,
    table_name="vector_store_table",
)

# Add documents
texts = ["foo", "bar", "baz", "boo"]
ids = [str(uuid.uuid4()) for i in range(len(texts))]
docs = [Document(page_content=texts[i]) for i in range(len(texts))]
await vs.aadd_documents(docs, ids=ids)

# Search documents
results = await vs.asimilarity_search("foo")